In [89]:
import pandas as pd
import pymysql
import jieba
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import naive_bayes
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
import tushare as ts
from snownlp import SnowNLP

In [2]:
# 获取京东方A股票的历史行情
ts.set_token("e3465973d838cdf0ce55a6d614c8a65bc45a2d99653fb87dfd718086")
pro = ts.pro_api()
df_725 = pro.daily(ts_code='000725.SZ', start_date='20100101', end_date='20191229')
df_725["trade_date"] = pd.to_datetime(df_725["trade_date"])
df_725 = df_725.drop(["ts_code", "open", "high", "low", "close", "pre_close", "pct_chg", "vol", "amount"], axis=1)
print(df_725.shape)
df_725.head(10)

(2413, 2)


,trade_date,change
0,2019-12-27,-0.19
1,2019-12-26,0.09
2,2019-12-25,0.04
3,2019-12-24,0.14
4,2019-12-23,-0.12
5,2019-12-20,-0.01
6,2019-12-19,0.03
7,2019-12-18,-0.09
8,2019-12-17,0.36
9,2019-12-16,0.28


In [3]:
# 从数据库获取文本数据
conn = pymysql.connect(host = "localhost",user = "root",passwd = "mysql",db = "mydata",charset="utf8")
sql_query = 'SELECT * FROM jdf_detail'
df = pd.read_sql(sql_query, con=conn)

conn.close()  #使用完后记得关掉

In [4]:
df.head()

,id,list_id,用户名,用户id,时间和客户端,发帖时间,客户端名称,标题,内容,来源,更新时间,创建时间
0,199,173243,财经评论,9313013693864916,发表于 2019-12-28 09:07:36 股吧网页版,2019-12-28 09:07:36,股吧网页版,新证券法明年3月1日起施行！全面推行注册制、加大违法成本 七大要点看过来,12月28日上午，十三届全国人大常委会第十五次会议全体会议审议通过了新修订的证券法。修订后的...,"http://guba.eastmoney.com/news,cjpl,894749996....",2019-12-29 14:14:03,2019-12-29 14:14:03
1,200,173244,财经评论,9313013693864916,发表于 2019-12-27 21:58:54 股吧网页版,2019-12-27 21:58:54,股吧网页版,超预期！新三板投资门槛全面下调 150万股民将跑步进场,2019年的最后一个周五下午，新三板市场迎来了最激动人心的时刻，深化改革真正落锤。,"http://guba.eastmoney.com/news,cjpl,894725732....",2019-12-29 14:14:03,2019-12-29 14:14:03
2,201,173245,财经评论,9313013693864916,发表于 2019-12-28 09:08:22 股吧网页版,2019-12-28 09:08:22,股吧网页版,央行：推进存量浮动利率贷款定价基准转换为LPR利率,,"http://guba.eastmoney.com/news,cjpl,894749886....",2019-12-29 14:14:03,2019-12-29 14:14:03
3,202,173246,财经评论,9313013693864916,发表于 2019-12-28 09:36:03 股吧网页版,2019-12-28 09:36:03,股吧网页版,证券法明确全面推行注册制,证券时报记者获悉，新修订的证券法明确全面推行注册制。将发行股票应当“具有持续盈利能力”的要求...,"http://guba.eastmoney.com/news,cjpl,894751250....",2019-12-29 14:14:03,2019-12-29 14:14:03
4,203,173247,东方财富网,6712111507146464,发表于 2019-07-17 11:05:48 股吧网页版,2019-07-17 11:05:48,股吧网页版,炒股理财选东方财富证券！7×24小时股票开户，佣金低至万2.5,None,"http://guba.eastmoney.com/news,cjpl,857016161....",2019-12-29 14:14:03,2019-12-29 14:14:03


In [39]:
# 过滤官方的帖子
df1 = df.loc[(df["用户名"] != "财经评论") & (df["用户名"] != "东方财富网") & (df["用户名"] != "京东方A资讯")]
# 删除无用的列
drop_list = ["id", "list_id", "用户名", "用户id", "时间和客户端", "客户端名称", "内容", "来源", "更新时间", "创建时间"]
df2 = df1.drop(drop_list, axis=1)

# 去掉缺失值
df2 = df2.dropna()
# 把发帖时间转为时间格式
df2['发帖时间'] = df2['发帖时间'].dt.strftime('%Y-%m-%d')
# 为方便后期聚合每日的帖子，这里为每一个帖子加上分号
df2["标题"] = df2["标题"] + ";"
# 按照时间排序
df2 = df2.sort_values(by="发帖时间", ascending=False)
# 重建索引
df2 = df2.reset_index()
# 删除旧索引列
df2 = df2.drop(["index"], axis=1)
df2.head(100)

,发帖时间,标题
0,2019-12-29,周一给昊华能源陪葬！;
1,2019-12-29,这下可彻底毁了，随着时间的临近怕是欺骗不下去了！;
2,2019-12-29,绿油油的韭菜地呦，一眼望不到边～我愿做其中的一小颗呦，此心无悔。;
3,2019-12-29,周一可以跌停了;
4,2019-12-28,娱乐互动：‘’娱乐互动‘’[俏皮][俏皮][俏皮]严禁高举‘’标语或口号‘’~摇;
5,2019-12-28,本月连出5份研报;
6,2019-12-28,京东方版花房姑娘我独自走过你身旁并没有话要对你讲我不敢抬头看着你的噢脸庞你问我要;
7,2019-12-28,消息面：不要过度纠结~‘’消息面‘’本质上的好坏(利空还是利多)[俏皮][俏皮];
8,2019-12-28,周末意淫~畅聊无罪[大笑][大笑][大笑]！！！;
9,2019-12-28,虽然跌的比预期想的多，不改看6，望10。自建仓近六百天，不改初衷。;


In [40]:
def get_sentiment(text):
    s = SnowNLP(text)
    r = s.sentiments
    return r

# 情感分析，会花比较久的时间
df2["sentiments"] = df2["标题"].apply(get_sentiment)
df2.head(100)

,发帖时间,标题,sentiments
0,2019-12-29,周一给昊华能源陪葬！;,0.060706
1,2019-12-29,这下可彻底毁了，随着时间的临近怕是欺骗不下去了！;,0.004522
2,2019-12-29,绿油油的韭菜地呦，一眼望不到边～我愿做其中的一小颗呦，此心无悔。;,0.017165
3,2019-12-29,周一可以跌停了;,0.142453
4,2019-12-28,娱乐互动：‘’娱乐互动‘’[俏皮][俏皮][俏皮]严禁高举‘’标语或口号‘’~摇;,0.296139
5,2019-12-28,本月连出5份研报;,0.555607
6,2019-12-28,京东方版花房姑娘我独自走过你身旁并没有话要对你讲我不敢抬头看着你的噢脸庞你问我要;,0.999057
7,2019-12-28,消息面：不要过度纠结~‘’消息面‘’本质上的好坏(利空还是利多)[俏皮][俏皮];,0.018229
8,2019-12-28,周末意淫~畅聊无罪[大笑][大笑][大笑]！！！;,0.679459
9,2019-12-28,虽然跌的比预期想的多，不改看6，望10。自建仓近六百天，不改初衷。;,0.000201


In [79]:
# 按照日期聚合每条帖子的情感评分，计算平均值，构造新的数据集
df2_sentiments = df2.groupby("发帖时间").mean()
df2_sentiments = df2_sentiments.reset_index()
df2_sentiments = df2_sentiments.rename(columns={"发帖时间": "trade_date"})
df2_sentiments['trade_date'] = pd.to_datetime(df2_sentiments['trade_date'], format='%Y-%m-%d')
df2_sentiments = df2_sentiments.sort_values(by="trade_date", ascending=False)
print(df2_sentiments.dtypes)
df2_sentiments.head()

trade_date    datetime64[ns]
sentiments           float64
dtype: object


,trade_date,sentiments
314,2019-12-29,0.056211
313,2019-12-28,0.515263
312,2019-12-27,0.485479
311,2019-12-26,0.503190
310,2019-12-25,0.524342


In [80]:
# 删除此数据集中的情感评分，便于聚合帖子内容
df2grp = df2.drop(["sentiments"], axis=1)
# 按照发帖时间将帖子内容拼接起来
df2grp = df2grp.groupby("发帖时间").sum()
# 去掉缺失值
df2grp = df2grp.dropna()
# 构造新列：当日帖子总长度
df2grp["len"] = df2grp["标题"].apply(len)
# 按照时间排序
df2grp = df2grp.sort_values(by="发帖时间", ascending=False)
# 重建索引
df2grp = df2grp.reset_index()
# 将时间转为日期类型
df2grp["trade_date"] = pd.to_datetime(df2grp["发帖时间"])
# 删除无用字段
df2grp = df2grp.drop(["发帖时间"], axis=1)
print(df2grp.shape)
df2grp.head(10)

(315, 3)


,标题,len,trade_date
0,周一给昊华能源陪葬！;这下可彻底毁了，随着时间的临近怕是欺骗不下去了！;绿油油的韭菜地呦，一...,77,2019-12-29
1,娱乐互动：‘’娱乐互动‘’[俏皮][俏皮][俏皮]严禁高举‘’标语或口号‘’~摇;本月连出5...,3743,2019-12-28
2,什么原因停牌;已出了五个多亿了，低于3.7严重破位下跌大家快跑吧;京东方作为最大最出名的散户...,13617,2019-12-27
3,这交易量创造历史了;完了，假突破！;出货速度真快，换装吗？还是散户力量大，全部接;上了半天上...,14381,2019-12-26
4,只需记住一点，主力不是来做解放军的;今天收盘多少钱？谁能猜对送你100股[大笑];抄底合力泰...,8984,2019-12-25
5,反弹与反转：勿把反弹~当作反转[俏皮][俏皮][俏皮]反弹~是反弹[俏皮][俏皮;站稳，坐好...,10183,2019-12-24
6,外资会是京东方股市中的董垂瑞吗;下午大盘翻红;又玄了;京东方本周上5元同意吗？[赞][赞][...,10721,2019-12-23
7,[图片];此股最高可达50元～～如果有钱可以走这股十倍返还利益;京东方必跌破1元退市！;垃圾...,3087,2019-12-22
8,告诫：‘’韭菜根‘’的~告诫[俏皮][俏皮][俏皮]对于我们散户投资者~股市投资;资本大鳄：...,2943,2019-12-21
9,不知不觉17个点了;能行的话，拉高出货吧;4.71买的京东方4.79做个踢;二次拉起才收到筹...,9498,2019-12-20


In [81]:
# 语料和股票走势连接
condf = pd.merge(df2grp,df_725,how='left',on="trade_date")
condf = condf.dropna()
condf.loc[condf["change"] <= 0,'change'] = 0
condf.loc[condf["change"] > 0,'change'] = 1
print(condf.shape)
condf.head(20)

(218, 4)


,标题,len,trade_date,change
2,什么原因停牌;已出了五个多亿了，低于3.7严重破位下跌大家快跑吧;京东方作为最大最出名的散户...,13617,2019-12-27,0.0
3,这交易量创造历史了;完了，假突破！;出货速度真快，换装吗？还是散户力量大，全部接;上了半天上...,14381,2019-12-26,1.0
4,只需记住一点，主力不是来做解放军的;今天收盘多少钱？谁能猜对送你100股[大笑];抄底合力泰...,8984,2019-12-25,1.0
5,反弹与反转：勿把反弹~当作反转[俏皮][俏皮][俏皮]反弹~是反弹[俏皮][俏皮;站稳，坐好...,10183,2019-12-24,1.0
6,外资会是京东方股市中的董垂瑞吗;下午大盘翻红;又玄了;京东方本周上5元同意吗？[赞][赞][...,10721,2019-12-23,0.0
9,不知不觉17个点了;能行的话，拉高出货吧;4.71买的京东方4.79做个踢;二次拉起才收到筹...,9498,2019-12-20,0.0
10,这才是真的龙头;今天量能如果再次超过一百亿元，股价必须新高，否则就有可能不妙！正常量价关系应...,12348,2019-12-19,1.0
11,大盘还得靠你啊你一下去大盘就完了这波行情也完了;明天我进来你慢慢跌;要涨停的;调整到5日线接...,13818,2019-12-18,0.0
12,傻呀！;卧槽，大象要2连板啊;加油！准备板;要涨停板？;太bug了;今天来个涨停[鼓掌];除...,24798,2019-12-17,1.0
13,神仙打架，我们看戏！;涨停卖不卖呢，纠结;无须扬鞭自奋蹄;做了点波段，现在等着这波行情赚点;...,20288,2019-12-16,1.0


In [82]:
# 语料和情感评分连接
condf = pd.merge(condf,df2_sentiments,how='left',on="trade_date")
print(condf.shape)
condf.head(20)

(218, 5)


,标题,len,trade_date,change,sentiments
0,什么原因停牌;已出了五个多亿了，低于3.7严重破位下跌大家快跑吧;京东方作为最大最出名的散户...,13617,2019-12-27,0.0,0.485479
1,这交易量创造历史了;完了，假突破！;出货速度真快，换装吗？还是散户力量大，全部接;上了半天上...,14381,2019-12-26,1.0,0.503190
2,只需记住一点，主力不是来做解放军的;今天收盘多少钱？谁能猜对送你100股[大笑];抄底合力泰...,8984,2019-12-25,1.0,0.524342
3,反弹与反转：勿把反弹~当作反转[俏皮][俏皮][俏皮]反弹~是反弹[俏皮][俏皮;站稳，坐好...,10183,2019-12-24,1.0,0.492017
4,外资会是京东方股市中的董垂瑞吗;下午大盘翻红;又玄了;京东方本周上5元同意吗？[赞][赞][...,10721,2019-12-23,0.0,0.464157
5,不知不觉17个点了;能行的话，拉高出货吧;4.71买的京东方4.79做个踢;二次拉起才收到筹...,9498,2019-12-20,0.0,0.497889
6,这才是真的龙头;今天量能如果再次超过一百亿元，股价必须新高，否则就有可能不妙！正常量价关系应...,12348,2019-12-19,1.0,0.526927
7,大盘还得靠你啊你一下去大盘就完了这波行情也完了;明天我进来你慢慢跌;要涨停的;调整到5日线接...,13818,2019-12-18,0.0,0.505621
8,傻呀！;卧槽，大象要2连板啊;加油！准备板;要涨停板？;太bug了;今天来个涨停[鼓掌];除...,24798,2019-12-17,1.0,0.509588
9,神仙打架，我们看戏！;涨停卖不卖呢，纠结;无须扬鞭自奋蹄;做了点波段，现在等着这波行情赚点;...,20288,2019-12-16,1.0,0.502670


In [83]:
# 分词函数，没有停用词
def xh_cut(intxt):
    return " ".join([w for w in jieba.cut(intxt)])

# 将语料分词
condf["cut"] = condf["标题"].apply(xh_cut)
print(condf.shape)
condf.head()

(218, 6)


,标题,len,trade_date,change,sentiments,cut
0,什么原因停牌;已出了五个多亿了，低于3.7严重破位下跌大家快跑吧;京东方作为最大最出名的散户...,13617,2019-12-27,0.0,0.485479,什么 原因 停牌 ; 已出 了 五个 多亿 了 ， 低于 3.7 严重 破位 下跌 大家 快...
1,这交易量创造历史了;完了，假突破！;出货速度真快，换装吗？还是散户力量大，全部接;上了半天上...,14381,2019-12-26,1.0,0.503190,这 交易量 创造 历史 了 ; 完 了 ， 假 突破 ！ ; 出货 速度 真 快 ， 换装 ...
2,只需记住一点，主力不是来做解放军的;今天收盘多少钱？谁能猜对送你100股[大笑];抄底合力泰...,8984,2019-12-25,1.0,0.524342,只 需 记住 一点 ， 主力 不是 来 做 解放军 的 ; 今天 收盘 多少 钱 ？ 谁 能...
3,反弹与反转：勿把反弹~当作反转[俏皮][俏皮][俏皮]反弹~是反弹[俏皮][俏皮;站稳，坐好...,10183,2019-12-24,1.0,0.492017,反弹 与 反转 ： 勿 把 反弹 ~ 当作 反转 [ 俏皮 ] [ 俏皮 ] [ 俏皮 ] ...
4,外资会是京东方股市中的董垂瑞吗;下午大盘翻红;又玄了;京东方本周上5元同意吗？[赞][赞][...,10721,2019-12-23,0.0,0.464157,外资 会 是 京东方 股市 中 的 董垂瑞 吗 ; 下午 大盘 翻红 ; 又 玄 了 ; 京...


## 训练朴素贝叶斯模型

In [97]:
# 词袋模型-矩阵
vect = CountVectorizer()
wordmtx = vect.fit_transform(condf["cut"])
print(wordmtx.shape)
# print(wordmtx)

(218, 34298)


In [98]:
# tfidf-矩阵
transf = TfidfTransformer()
tfidf = transf.fit_transform(wordmtx)
# print(tfidf)
print(tfidf.todense().shape)
# print(vect.vocabulary_)

(218, 34298)


In [99]:
# 训练朴素贝叶斯模型
x_train, x_test, y_train, y_test = train_test_split(wordmtx, condf["change"], test_size=0.3, random_state=30)
nbmodel = naive_bayes.MultinomialNB()
nbmodel.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [100]:
# 预测
y_predict = nbmodel.predict(x_test)
# 预测准确率
train_score = nbmodel.score(x_train, y_train)
test_score = nbmodel.score(x_test, y_test)
print("训练集准确率：{}, 测试集准确率：{}".format(train_score, test_score))

训练集准确率：0.9736842105263158, 测试集准确率：0.7121212121212122


## 训练逻辑回归模型

In [93]:
condf_sentiments = pd.DataFrame(condf["sentiments"])
condf_sentiments.head()

,sentiments
0,0.485479
1,0.503190
2,0.524342
3,0.492017
4,0.464157


In [94]:
# 训练逻辑回归模型
x_train, x_test, y_train, y_test = train_test_split(condf_sentiments, condf["change"], test_size=0.3, random_state=30)
logic = LogisticRegression(penalty="l2", C=1)
logic.fit(x_train, y_train)

/Users/nick/.venv/ML-training/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [95]:
# 预测
y_predict = logic.predict(x_test)
# 预测准确率
train_score = logic.score(x_train, y_train)
test_score = logic.score(x_test, y_test)
print("训练集准确率：{}, 测试集准确率：{}".format(train_score, test_score))

训练集准确率：0.5460526315789473, 测试集准确率：0.6060606060606061
